In [1]:
from dash import Dash, dcc, html, dash_table
import dash_leaflet as dl
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.express as px
import base64
import os
import numpy as np
import pandas as pd
from flask import Flask, session, redirect, url_for, request
from flask_bcrypt import Bcrypt
from flask_pymongo import PyMongo
from functools import wraps
from mongoCRUD import MongoDBHelper

# Flask App for Authentication and MongoDB Connection
server = Flask(__name__)
server.secret_key = "secret_key"  # I plan to replace with a secure secret key
server.config["MONGO_URI"] = "mongodb://localhost:27017/shelter_database"
mongo = PyMongo(server)
bcrypt = Bcrypt(server)

# Role-Based Access Control Decorator
def role_required(role):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if "user_role" not in session or session["user_role"] != role:
                return html.Div("Access denied: Insufficient permissions."), 403
            return func(*args, **kwargs)
        return wrapper
    return decorator

# Dash App Setup
app = Dash(__name__, server=server)
mongo_helper = MongoDBHelper(db_name="shelter_database", collection_name="outcomes")

# Load Data
csv_file_path = 'aac_shelter_outcomes.csv'
if mongo_helper.collection.count_documents({}) == 0:
    mongo_helper.import_csv_to_mongo(csv_file_path)
df = pd.DataFrame.from_records(mongo_helper.fetch_all_records())
df.drop(columns=['_id'], inplace=True)
columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

# Encode Logo
image_filename = 'Grazioso-Salvare-Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App Layout
app.layout = html.Div([  
    html.Center(html.A(
        href='https://snhu.edu',
        target='_blank',
        children=[html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'width': '10%'}
        )]
    )),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.P('Colin Aheron'))),
    dcc.Input(id='username', type='text', placeholder='Enter Username'),
    dcc.Input(id='password', type='password', placeholder='Enter Password'),
    html.Button('Login', id='login-button', n_clicks=0),
    html.Div(id='login-output'),
    dcc.Input(id='register-username', type='text', placeholder='New Username'),
    dcc.Input(id='register-password', type='password', placeholder='New Password'),
    dcc.Dropdown(id='register-role', options=[
        {'label': 'Admin', 'value': 'Admin'},
        {'label': 'Analyst', 'value': 'Analyst'},
        {'label': 'Viewer', 'value': 'Viewer'}
    ], placeholder='Select Role'),
    html.Button('Register', id='register-button', n_clicks=0),
    html.Div(id='register-output'),
    html.Div(id='dashboard-content', style={'display': 'none'}, children=[  # Dashboard layout...
        dcc.Dropdown(['Reset', 
                      'Water Rescue', 
                      'Mountain or Wilderness Rescue',
                      'Disaster or Individual Tracking'], 'Select Rescue Type', id='dropdown'),
        html.Hr(),
        dash_table.DataTable(id='datatable-id',
                             columns=columns,
                             data=df.to_dict('records'),
                             page_size=10,
                             sort_action='native',
                             row_selectable="single"),
        html.Br(),
        html.Div([html.Button('Create', id='create-button', n_clicks=0),
                  html.Button('Update', id='update-button', n_clicks=0),
                  html.Button('Delete', id='delete-button', n_clicks=0)],
                 id='crud-buttons', style={'display': 'flex', 'gap': '10px'}),
        html.Div(className='row', style={'display': 'flex'}, children=[  # Graph and Map layout...
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ])
    ])
])

# Function to register a new user
@app.callback(
    Output('register-output', 'children'),
    [Input('register-button', 'n_clicks')],
    [State('register-username', 'value'), State('register-password', 'value'), State('register-role', 'value')]
)
def register_user(n_clicks, username, password, role):
    if n_clicks > 0:
        # Encrypt password before storing it
        hashed_password = bcrypt.generate_password_hash(password).decode('utf-8')
        
        # Sanitize input (preventing injection)
        if not username or not password or not role:
            return "All fields are required!"
        
        # Check if username already exists in the database
        existing_user = mongo.db.users.find_one({"username": username})
        if existing_user:
            return "Username already exists!"

        # Store the user in the database with the hashed password
        mongo.db.users.insert_one({"username": username, "password": hashed_password, "role": role})
        return f"User {username} registered successfully!"

    return ""

# Callback for login functionality with password verification
@app.callback(
    [Output('login-output', 'children'),
     Output('dashboard-content', 'style'),
     Output('crud-buttons', 'style')],
    [Input('login-button', 'n_clicks')],
    [State('username', 'value'), State('password', 'value')]
)
def login_user(n_clicks, username, password):
    if n_clicks > 0:
        # Sanitize inputs (e.g., escape characters in the username)
        if not username or not password:
            return "Both fields are required.", {'display': 'none'}, {'display': 'none'}
        
        # Look up the user from the database
        user = mongo.db.users.find_one({"username": username})
        
        # Verify user and password
        if user and bcrypt.check_password_hash(user['password'], password):
            session['username'] = username
            session['user_role'] = user['role']
            role = user['role']
            if role == 'Admin':
                return f"Welcome {session['username']}! Role: {role}", {'display': 'block'}, {'display': 'flex'}
            elif role == 'Analyst':
                return f"Welcome {session['username']}! Role: {role}", {'display': 'block'}, {'display': 'none'}
            elif role == 'Viewer':
                return f"Welcome {session['username']}! Role: {role}", {'display': 'block'}, {'display': 'none'}
        else:
            return "Invalid credentials. Please try again.", {'display': 'none'}, {'display': 'none'}
    
    return "", {'display': 'none'}, {'display': 'none'}

# Callback for updating dashboard data (including map and graph)
@app.callback(
    [Output('datatable-id', 'data'),
     Output('graph-id', 'children'),
     Output('map-id', 'children')],
    [Input('dropdown', 'value'),
     Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_dashboard(value, derived_virtual_data, selected_rows):
    # Handle data table update
    df_filtered = df
    if value == 'Water Rescue':
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        df_filtered = df[df['breed'].isin(breeds)]
    elif value == 'Mountain or Wilderness Rescue':
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
        df_filtered = df[df['breed'].isin(breeds)]
    elif value == 'Disaster or Individual Tracking':
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        df_filtered = df[df['breed'].isin(breeds)]

    # Graph update
    graph = dcc.Graph(figure=px.pie(df_filtered, names='breed', title='Preferred Animals'))

    # Map update
    if not selected_rows:
        raise PreventUpdate
    
    row = selected_rows[0]
    data_for_map = df_filtered.iloc[row]
    
    # Check if the latitude and longitude are valid
    if pd.isnull(data_for_map['location_lat']) or pd.isnull(data_for_map['location_long']):
        return df_filtered.to_dict('records'), graph, html.Div("Invalid location data")

    # Valid lat/lng, proceed with map rendering
    map_view = dl.Map(style={'width': '1000px', 'height': '500px'},
                      center=[data_for_map['location_lat'], data_for_map['location_long']], zoom=10,
                      children=[dl.TileLayer(),
                                dl.Marker(position=[data_for_map['location_lat'], data_for_map['location_long']]),
                                dl.Tooltip(data_for_map['breed']),
                                dl.Popup([html.H1("Animal Name:"), html.H1(data_for_map['name'])])])

    return df_filtered.to_dict('records'), graph, map_view

# Run the Server
if __name__ == "__main__":
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [ ]:
%pip install dash
%pip install dash-leaflet
%pip install dash-table
%pip install plotly
%pip install pandas
%pip install matplotlib
%pip install Flask
%pip install Flask-Bcrypt
%pip install Flask-PyMongo


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
